In [1]:
import os

def import_from_directory(directory, module_name):
    current_dir = os.getcwd()
    os.chdir(directory)
    module = __import__(module_name)
    os.chdir(current_dir)
    return module

test_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'test_base')
model_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'model_base')

2024-05-10 10:04:46.508214: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 10:04:46.547554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 10:04:46.547592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 10:04:46.548727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 10:04:46.555476: I tensorflow/core/platform/cpu_feature_guar

In [2]:
SEED = 42
model_base.set_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/notebooks/'
os.chdir(desired_directory)

In [3]:
import pandas as pd

In [7]:
data_path = 'Data/test_data/humancohort_data.xlsx'
df_pos = pd.read_excel(data_path, sheet_name='positive', header = 4)[['Alignment ID','Average Mz', 'Metabolite name', 'Adduct type', 'MS/MS assigned','Ontology', 'INCHIKEY', 'SMILES', 'MS/MS spectrum']]
df_pos2 = df_pos[~df_pos['MS/MS spectrum'].isna()].reset_index(drop=True).\
    rename(columns={'Metabolite name':'Metabolitename', 'Alignment ID':'AlignmentID','Average Mz':'AverageMz', 'MS/MS spectrum':'MSMSspectrum', 'Adduct type':'Adducttype'})
df_pos2['datasetID'] = 1

In [8]:
df_pos3 = model_base.cal_df_sel_column(df_pos2)
df_pos_wide = model_base.cal_wide_df(df_pos3)

In [9]:
column_path = 'Data/model_data/pos/posmodelcolumn_all.pkl'
replacement_dict_path = 'Data/model_data/pos/pos_replacement_dict_all.pkl'
model_path = "models/pos_class_model"

replacement_dict, loaded_columns, loaded_model =\
        test_base.load_models(df_pos_wide, replacement_dict_path, column_path, model_path)

df_pos_wide2, columns_to_drop, replacement_dict_reversed, replacement_dict_reversed_df =\
        test_base.pred_data_adj(df_pos_wide, replacement_dict, loaded_columns)

df_test_predclass, df_test_predclass_v, df_pos_wide2, y_test_acc_, y_pred_test_max_ =\
        test_base.pred_data_by_model(df_pos_wide2, replacement_dict, replacement_dict_reversed, replacement_dict_reversed_df, loaded_model, columns_to_drop)

576/576 [==============================] - 4s 6ms/step


In [ ]:
df_pos_wide_s = df_pos_wide[['Metabolitename','Ontology', 'AverageMz', 'Adducttype','ID']].reset_index().rename(columns={'index':'idx'})

df_test_predclass_v2 = df_pos_wide_s.merge(df_test_predclass_v, on='idx', how='inner')
df_test_predclass_v2 = df_test_predclass_v2.dropna()
df_test_predclass_v2 = df_test_predclass_v2[~df_test_predclass_v2['Metabolitename'].str.contains("w/o MS2")] #w/o MS2|low score

In [ ]:
df_test_predclass_v2['pred_class_list'] = test_base.make_pred_class_list_column(df_test_predclass_v2)
df_test_predclass_v2['TF'] = df_test_predclass_v2.apply(test_base.check_ontology_in_pred_class, axis=1)

GDCAE

In [11]:
import matplotlib.pyplot as plt

In [10]:
def extract_x(row):
    return row['ID'].split('_')[1] if '_' in row['ID'] else None

def plot_spectrum(df_wide, Alignmnentid = '8864'):
    
    df_wide.index = df_wide.apply(extract_x, axis=1)
    columns_to_drop = [ 'Metabolitename','Ontology', 'AverageMz', 'Adducttype','ID', 'EOvalue',	'MCHvalue']
    df_wide = df_wide.drop(columns=columns_to_drop).astype(float)
    df1= pd.DataFrame(df_wide.loc[Alignmnentid]).reset_index().rename(columns={'index':'mz',Alignmnentid:'exp'}).astype('float')

    fig, axs = plt.subplots(1, 1, figsize=(4, 3))

    axs.plot(df1['mz'], df1['exp'], color='black')
    axs.set_xlabel('m/z', fontsize=12)
    axs.set_ylabel('Intensity', fontsize=12)
    axs.set_title('Alignment ID: ' + str(Alignmnentid), fontsize=12)
    axs.set_xlim(0, 1250)

    plt.show()

In [12]:
plot_spectrum(df_pos_wide, Alignmnentid = '16944')